# Semantic reranking with a Hugging Face model

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/11-semantic-reranking-hugging-face.ipynb)

In this notebook you'll learn how to implement semantic reranking in Elasticsearch by uploading a model from Hugging Face into your cluster. You'll also learn about the `retriever` abstraction, a simpler syntax for crafting queries and combining different search operations.

You will:

- Choose a cross-encoder model from Hugging Face to perform semantic reranking
- Upload the model to your Elasticsearch deployment using [Eland](https://www.elastic.co/guide/en/elasticsearch/client/eland/current/machine-learning.html)
- Create an inference endpoint to manage your `rerank` task
- Query your data using the `text_similarity_rerank` retriever

## 🧰 Requirements

For this example, you will need:

- An Elastic deployment:
    
    - We'll be using [Elastic Cloud](https://www.elastic.co/guide/en/cloud/current/ec-getting-started.html) for this example (available with a [free trial](https://cloud.elastic.co/registration?onboarding_token=vectorsearch&utm_source=github&utm_content=elasticsearch-labs-notebook))
      - You can also use Elastic Cloud [Serverless](https://cloud.elastic.co/serverless-registration)
- Elasticsearch 8.15.0 or above (for non-serverless deployments)

## Install packages


In [2]:
!pip install -qU elasticsearch
!pip install eland[pytorch]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.2/480.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 4.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Usi

## Import packages

In [3]:
import os
from elasticsearch import Elasticsearch, helpers, exceptions
from urllib.request import urlopen
from getpass import getpass
import json
import time

## Initialize Elasticsearch Python client

You need to connect to a running Elasticsearch instance. In this example we're using an Elastic Cloud deployment.

In [4]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# Create the client instance
client = Elasticsearch(
    # For local development
    # hosts=["http://localhost:9200"]
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
)

Elastic Cloud ID: ··········
Elastic Api Key: ··········


## Test connection

Confirm that the Python client has connected to your Elasticsearch instance with this test.



In [70]:
print(client.info())

{'name': 'serverless', 'cluster_name': 'dbed854f18874014ad9ca1b1169e9e3d', 'cluster_uuid': 't3JzgD0aSeesEYPWch7KUg', 'version': {'number': '8.11.0', 'build_flavor': 'serverless', 'build_type': 'docker', 'build_hash': '00000000', 'build_date': '2023-10-31', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '8.11.0', 'minimum_index_compatibility_version': '8.11.0'}, 'tagline': 'You Know, for Search'}


## Enable Telemetry

Knowing that you are using this notebook helps us decide where to invest our efforts to improve our products. We would like to ask you that you run the following code to let us gather anonymous usage statistics. See telemetry.py for details. Thank you!

In [ ]:
!curl -O -s https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/telemetry/telemetry.py
from telemetry import enable_telemetry

es_client = enable_telemetry(es_client, "11-semantic-reranking-hugging-face")

## Upload sample data

This examples uses a small dataset of movies.

In [21]:
url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/movies.json"
response = urlopen(url)

# Load the response data into a JSON object
data_json = json.loads(response.read())

# Prepare the documents to be indexed
documents = []
for doc in data_json:
    documents.append(
        {
            "_index": "movies",
            "_source": doc,
        }
    )

# Use helpers.bulk to index
helpers.bulk(client, documents)

print("Done indexing documents into `movies` index!")
time.sleep(3)

Done indexing documents into `movies` index!


## Upload Hugging Face model using Eland

Now we'll use Eland's `eland_import_hub_model` command to upload the model to Elasticsearch. For this example we've chosen the [`cross-encoder/ms-marco-MiniLM-L-6-v2`](https://huggingface.co/cross-encoder/ms-marco-MiniLM-L-6-v2) text similarity model.

**Refer to [the Elastic NLP model reference](https://www.elastic.co/guide/en/machine-learning/8.15/ml-nlp-model-ref.html#ml-nlp-model-ref-text-similarity) for a list of third-party text similarity models supported by Elasticsearch.**

ℹ️
This example uses an Elastic Cloud deployment with an API key, but there are more deployment and [authentication options](https://www.elastic.co/guide/en/elasticsearch/client/eland/current/machine-learning.html#ml-nlp-pytorch-auth).

In [6]:
!eland_import_hub_model \
  --cloud-id $CLOUD_ID \
  --es-api-key $ES_API_KEY \
  --hub-model-id cross-encoder/ms-marco-MiniLM-L-6-v2 \
  --task-type text_similarity \
  --clear-previous \
  --start

2024-08-13 06:47:43,002 INFO : Establishing connection to Elasticsearch
2024-08-13 06:47:43,550 INFO : Connected to serverless cluster 'dbed854f18874014ad9ca1b1169e9e3d'
2024-08-13 06:47:43,551 INFO : Loading HuggingFace transformer tokenizer and model 'cross-encoder/ms-marco-MiniLM-L-6-v2'
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
tokenizer_config.json: 100% 316/316 [00:00<00:00, 1.76MB/s]
config.json: 100% 794/794 [00:00<00:00, 4.32MB/s]
vocab.txt: 100% 232k/232k [00:00<00:00, 15.2MB/s]
special_tokens_map.json: 100% 112/112 [00:00<00:00, 610kB/s]
pytorch_model.bin: 100% 90.9M/90.9M [00:05<00:00, 16.2MB/s]
STAGE:2024-08-13 06:47:54 1405:1405 ActivityProfilerController.cpp:312] Completed Stage: Warm Up
STAGE:2024-08-13 06:47:54 1405:1405 ActivityPr

## Lexical queries

First let's use a `standard` retriever to test out some lexical (or full-text) searchs and then we'll compare the improvements when we layer in semantic reranking.

### Lexical match with `query_string` query

Let's say we vaguely remember that there is a famous movie about a killer who eats his victims. For the sake of argument, pretend we've momentarily forgotten the word "cannibal".

Let's perform a [`query_string` query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html) to find the phrase "flesh-eating bad guy" in the `plot` fields of our Elasticsearch documents.

In [44]:
resp = client.search(
    index="movies",
    retriever={
        "standard": {
            "query": {
                "query_string": {
                    "query": "flesh-eating bad guy",
                    "default_field": "plot",
                }
            }
        }
    },
)

if resp["hits"]["hits"]:
    for hit in resp["hits"]["hits"]:
        title = hit["_source"]["title"]
        plot = hit["_source"]["plot"]
        print(f"Title: {title}\nPlot: {plot}\n")
else:
    print("No search results found")

No search results found


No results! Unfortunately we don't have any near exact matches for "flesh-eating bad guy". Because we don't have any more specific information about the exact phrasing in the Elasticsearch data, we'll need to cast our search net wider.

### Simple `match` query

This lexical query performs a standard keyword search for the term "crime" within the "plot" and "genre" fields of our Elasticsearch documents.

In [69]:
resp = client.search(
    index="movies",
    retriever={
        "standard": {
            "query": {"multi_match": {"query": "crime", "fields": ["plot", "genre"]}}
        }
    },
)

for hit in resp["hits"]["hits"]:
    title = hit["_source"]["title"]
    plot = hit["_source"]["plot"]
    print(f"Title: {title}\nPlot: {plot}\n")

Title: The Godfather
Plot: An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.

Title: Goodfellas
Plot: The story of Henry Hill and his life in the mob, covering his relationship with his wife Karen Hill and his mob partners Jimmy Conway and Tommy DeVito in the Italian-American crime syndicate.

Title: The Dark Knight
Plot: When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.

Title: Pulp Fiction
Plot: The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption.

Title: The Silence of the Lambs
Plot: A young F.B.I. cadet must receive the help of an incarcerated and manipulative cannibal killer to help catch another serial killer, a madman who skins his victims.

Title: The Usual Suspects
Plot: A sole survivor te

That's better! At least we've got some results now. We broadened our search criteria to increase the chances of finding relevant results.

But these results aren't very precise in the context of our original query "flesh-eating bad guy". We can see that "The Silence of the Lambs" is returned in the middle of the results set with this generic `match` query. Let's see if we can use our semantic reranking model to get closer to the searcher's original intent.

## Semantic reranker

In the following `retriever` syntax, we wrap our standard `match` query retriever in a `text_similarity_reranker`. This allows us to leverage the NLP model we deployed to Elasticsearch to rerank the results based on the phrase "flesh-eating bad guy".

In [68]:
resp = client.search(
    index="movies",
    retriever={
        "text_similarity_reranker": {
            "retriever": {
                "standard": {
                    "query": {
                        "multi_match": {"query": "crime", "fields": ["plot", "genre"]}
                    }
                }
            },
            "field": "plot",
            "inference_id": "my-msmarco-minilm-model",
            "inference_text": "flesh-eating bad guy",
        }
    },
)

for hit in resp["hits"]["hits"]:
    title = hit["_source"]["title"]
    plot = hit["_source"]["plot"]
    print(f"Title: {title}\nPlot: {plot}\n")

Title: The Silence of the Lambs
Plot: A young F.B.I. cadet must receive the help of an incarcerated and manipulative cannibal killer to help catch another serial killer, a madman who skins his victims.

Title: Pulp Fiction
Plot: The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption.

Title: Se7en
Plot: Two detectives, a rookie and a veteran, hunt a serial killer who uses the seven deadly sins as his motives.

Title: Goodfellas
Plot: The story of Henry Hill and his life in the mob, covering his relationship with his wife Karen Hill and his mob partners Jimmy Conway and Tommy DeVito in the Italian-American crime syndicate.

Title: The Dark Knight
Plot: When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.

Title: The Godfather
Plot: An organized crime dynasty's aging

Success! "The Silence of the Lambs" is our top result. Semantic reranking helped us find the most relevant result by parsing a natural language query, overcoming the limitations of lexical search that relies on keyword matching.

Semantic reranking enables semantic search in a few steps, without the need for generating and storing embeddings. This a great tool for testing and building hybrid search systems in Elasticsearch.

## Learn more

- [Semantic Reranking overview](https://www.elastic.co/guide/en/elasticsearch/reference/current/semantic-reranking.html). A high level overview of semantic reranking in Elasticsearch.
- [`text_similarity_reranker` retriever ](https://www.elastic.co/guide/en/elasticsearch/reference/current/retriever.html#text-similarity-reranker-retriever). Detailed API syntax reference with examples.
- [Semantic reranking with Cohere notebook](https://github.com/elastic/elasticsearch-labs/blob/main/notebooks/search/10-semantic-reranking-retriever-cohere.ipynb). You can also use the Cohere Rerank API instead of loading your own model into Elasticsearch.
- [`elasticsearch-labs` notebooks](https://github.com/elastic/elasticsearch-labs/tree/main/notebooks). Check our full catalogue of Python notebooks.